### 选股思路

1. 前天下跌大于-3.5%，越低越好
2. 今天高开，2-4%，接近2%最好
3. 爆量？行业？概念 集体高开？

In [68]:
import time
from datetime import datetime
import pandas as pd
import akshare as ak

def get_trade(symbol):
    """
    获取行业信息
    :param symbol:
    :return:
    """
    df = ak.stock_individual_info_em(symbol=symbol)
    dct = dict(zip(df['item'], df['value']))
    return dct['行业'], dct['股票简称']

ndf = pd.read_csv('concept.csv')

ndf.head()

,代码,名称,现价,涨跌幅,涨跌,涨速,换手,量比,振幅,成交额,流通股,流通市值,市盈率,概念
0,000001,平安银行,12.53,-1.18,-0.15,-0.16,0.45,1.16,2.05,10.92亿,194.06亿,2431.52亿,5.34,"证金持股,跨境支付（CIPS）,标普道琼斯A股,MSCI概念,深股通,融资融券"
1,000002,万科A,15.24,-2.31,-0.36,0.07,1.39,1.89,3.21,20.71亿,97.17亿,1480.82亿,8.04,"REITs概念,同花顺漂亮100,物业管理,装配式建筑,标普道琼斯A股,冰雪产业,超级品牌,..."
2,000004,ST国华,9.9,1.75,0.17,0.1,0.74,0.45,1.85,0.09亿,1.19亿,11.77亿,--,"电力物联网,数据安全,鸿蒙概念,网络安全,车联网,壳资源,华为概念,ST板块,5G,物联网"
3,000005,ST星源,1.77,1.14,0.02,0.0,0.43,0.89,2.29,0.08亿,10.58亿,18.73亿,139.53,"土壤修复,广东自贸区,ST板块,固废处理"
4,000006,深振业A,5.08,0.79,0.04,0.0,0.92,0.56,1.19,0.63亿,13.50亿,68.58亿,25.58,"证金持股,国企改革,深圳国企改革,物业管理,标普道琼斯A股,深股通,粤港澳大湾区,融资融券"


In [92]:
import os

count = 0
times_of_unusual = 5
folder = 'daily_data/'

files = os.listdir(folder)

data = []

for f in files:
    if not f.endswith('.csv'):
        continue

    code = f.split('.')[0]
    df = pd.read_csv(os.path.join(folder, f), encoding='utf-8')
    df['open_range'] = (df['开盘'] - df['昨收'])/df['昨收'] * 100
    df['open_range'] = df['open_range'].round(2)
    df['最高涨幅'] = (df['最高'] - df['昨收'])/df['昨收'] * 100
    df['最高涨幅'] = df['最高涨幅'].round(2)
    
    df['flag'] = (df['涨跌幅'].shift(1) <= -3.5) & (1.8 <= df['open_range']) & (df['open_range'] <= 4)
    df['yesterday_range'] = df['涨跌幅'].shift(1)
    df['tomorrow_range'] = df['涨跌幅'].shift(-1)
    df['tomorrow_open'] = df['open_range'].shift(-1)
    df['tomorrow_high'] = df['最高涨幅'].shift(-1)
    
    for idx, ser in df.iterrows():
        dct = ser.to_dict()
        if dct['flag']:
            row = {}
            n_ndf = ndf[ndf['代码'].isin([code])]
            if n_ndf.empty:
                continue
            first_row = n_ndf.iloc[0]
            first_dct = first_row.to_dict()
            
            if 'st' in first_dct['名称'].lower():
                continue
            
            # 趋势是好的，10日线大于20日线
            if dct['ma_10'] < dct['ma_20']*0.98:
                continue
            
            # 破二十日线也不考虑
            if dct['昨收'] < dct['ma_20']*0.98:
                continue
            
            # 流通市值太小的不考虑，次新股不考虑
            if float(first_dct["流通市值"][:-1]) < 20:
                continue
            if '次新' in first_dct['概念']:
                continue
            
            
            row['代码'] = code
            row['日期'] = dct['日期']
            row['月份'] = dct['日期'][:-3]
            
            row['名称'] = first_dct['名称']
            row['流通市值'] = first_dct["流通市值"]
            
            row['概念'] = first_dct['概念']
            
            row['昨收'] = dct['yesterday_range']
            row['开盘'] = dct['open_range']
            row['今收'] = dct['涨跌幅']
            row['明开'] = dct['tomorrow_open']
            row['明收'] = dct['tomorrow_range']
            
            row['开盘卖收益'] = row['今收'] - row['开盘'] + row['明开']
            row['收盘卖收益'] = row['今收'] - row['开盘'] + row['明收']
            row['最高卖收益'] = row['今收'] - row['开盘'] + dct['tomorrow_high']
            data.append(row.copy())

    count += 1
    # if count >= 100:
        # break
    
# 统计第三天卖出收益
# 以及每天交易频率 数量等

# 10日线大于20日线？
# 最低点以及 距离均线点位

# 靠近10日线比较好？支撑点强
# 连续大涨，见顶的不考虑
# 距离20日线太远

# 涨停板就不卖？
# 高开后回踩线？

# 高级买点： 低点买 或者高点买，或者支撑点（均线、昨天低点、收盘等位置）
# 连续大盘不好的时候，第二天算不算？？？

# 前面连续大涨的，没有消化获利盘的 不要碰 （最近半个月内连续涨停板的remove）
# 流通市值太小的 不考虑
# -3.5需要阴线？

In [ ]:
from IPython.display import display
mdf = pd.DataFrame(data)

tmp = None 
count1 = 0
count2 = 0
for idx, sdf in list(mdf.groupby('月份'))[::-1]:
    ct1 = round(sdf['开盘卖收益'].mean(), 2)
    ct2 = round(sdf['收盘卖收益'].mean(), 2)
    ct3 = round(sdf['最高卖收益'].mean(), 2)
    print('开盘卖收益:', ct1)
    print('收盘卖收益:', ct2)
    print('最高卖收益:', ct3)
    
    dct = sdf.iloc[0].to_dict()
    count1 += ct1
    count2 += ct2
    if tmp is None:
        tmp = dct['月份']
    
    if tmp and tmp != dct['月份']:
        tmp = dct['月份']
        print(tmp)
        print('开盘卖月收益:', count1)
        print('收盘卖月收益:', count2)
        count1 = 0
        count2 = 0
    
    display(sdf)

## 服务器脚本整理

In [146]:
# save last day history
import os
import time
import json
from datetime import datetime
import pandas as pd
import akshare as ak

import ssl
import smtplib
from email.mime.text import MIMEText
from email.header import Header
from email.message import EmailMessage
from email.mime.multipart import MIMEMultipart

def get_text():

    data = []
    folder = 'daily_data/'

    concept = pd.read_csv('concept.csv')

    cdf = ak.stock_zh_a_spot_em()
    # 只要主板的票
    cdf = cdf[cdf['代码'].str.startswith('00') | cdf['代码'].str.startswith('60')]
    # 过滤st
    cdf['名称'] = cdf['名称'].str.lower()
    cdf = cdf[~cdf['名称'].str.contains('st')]
    # 开盘涨幅
    cdf['open_range'] = (df['今开'] - df['昨收'])/df['昨收'] * 100
    cdf['open_range'] = cdf['open_range'].round(2)
    # 开盘涨幅filter
    cdf = cdf[(1.8 <= cdf['open_range']) & (cdf['open_range'] <= 4)]

    for idx, ser in cdf.iterrows():
        row = {}
        dct = ser.to_dict()
        code = dct['代码']
        ndf = pd.read_csv(os.path.join(folder, str(code)+'.csv'))
        last_dct = ndf.iloc[-2].to_dict()

        concept_df = concept[concept['代码'].isin([code])]
        if concept_df.empty:
            continue
        concept_dct = concept_df.iloc[0].to_dict()

        # 次新股不考虑
        if '次新' in concept_dct['概念']:
            continue

        if 'st' in dct['名称'].lower():
            continue

        # 趋势是好的，10日线大于20日线
        if last_dct['ma_10'] < last_dct['ma_20']*0.98:
            continue

        # 破二十日线也不考虑
        if last_dct['昨收'] < last_dct['ma_20']*0.98:
            continue

        # 流通市值太小的不考虑
        if float(concept_dct["流通市值"][:-1]) < 20:
            continue

        # 昨天收盘需要小于-3.45
        if not (last_dct['涨跌幅'] <= -3.45):
            continue

        row['代码'] = code
        # row['日期'] = last_dct['日期']
        row['名称'] = dct['名称']
        row['昨收'] = last_dct['涨跌幅']
        row['开盘'] = dct['open_range']
        row['流通市值'] = concept_dct["流通市值"]
        row['概念'] = concept_dct['概念']

        data.append(row)

    # pd.DataFrame(data)
    text = f'{datetime.now()}\n'
    for row in data:
        text += '-'*25 + '\n'
        for k, v in row.items():
            text += f'{k} : {v}\n'
    return text

def send_mail(text):
    key = 'letadehhnxtpbdca'
    EMAIL_ADDRESS = 'wikizeros@qq.com'
    EMAIL_PASSWORD = key
    context = ssl.create_default_context()
    sender = EMAIL_ADDRESS
    receiver = ['vance.coder@139.com']

    subject = datetime.now().strftime('%Y%m%d%H%M%S')
    msg = EmailMessage()
    msg['subject'] = subject
    msg['From'] = sender
    msg['To'] = receiver
    msg.set_content(text)

    # msg.attach(MIMEText(html_msg, "html", "utf-8"))

    with smtplib.SMTP_SSL("smtp.qq.com", 465, context=context) as smtp:
        # smtp.set_debuglevel(1)
        smtp.ehlo('smtp.qq.com')
        smtp.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
        smtp.send_message(msg)

text = get_text()
send_mail(text)
